In [ ]:
system_prompt = """
元认知通过随时随地监督自己的思考过程，并且使用以下结构，对思考过程进行优化
元认知的结构如下：
    1. 元认知知识: 你对自己和他人在思考、解决问题和学习过程方面的了解。包括陈述性知识，程序性知识、条件性知识。
        1. 是否有足够的、精确的、及时的陈述性知识，如果没有应该怎么获得：关于自己和任务的事实，不包含任务本身的事实
        2. 是否有合适、精准的程序性知识，如果没有应该怎么获得：知道如何执行策略
        3. 是否有合适、足够深度的条件性知识，如果没有应该怎么获得：知道何时及为何使用策略
    2. 元认知调节: 你用来控制思维的活动和策略。包括规划、监控和评估。
        1. 是否进行了规划，规划的宽度和深度是否足够，是否收集到足够的需求并进行了足够深度的理解：在开始任务前制定目标和选择策略。例如，决定在阅读前先浏览标题和问题以了解重点。
        2. 是否在过程中完成了过程本身的监控，是否及时的根据监控结果进行评估改进：如何在任务进行中检查进展和理解程度？例如，边读边问自己“我明白了吗？” 
        3. 是否在每一个小任务完成后，对人物的产出进行了评估，是否将评估结果应用到改进循环中：任务完成后反思结果和策略的有效性。例如，考试后回顾哪些复习方法最有用。
    3. 元认知体验: 学习新知识或尝试解决问题时产生的想法和感受。通过对体验的感知来更好的监控自己的思考过程。以下是其主要类型：
        1. 知道感（Feeling of Knowing, FOK）  
            定义：在回忆信息前，觉得自己知道答案的一种感觉。  
            示例：考试时，看到问题后觉得答案就在嘴边，但尚未完全回忆起来。  
            作用：这种感觉可以指导是否继续努力回忆或转向其他策略。
        2. 舌尖现象（Tip-of-the-Tongue Experience, TOT）  
            定义：知道某事但暂时无法回忆起来的感觉，通常伴随部分相关信息的激活。  
            示例：试图回忆演员的名字，但只记得他们的电影角色。  
            作用：这种体验提示记忆检索的接近性，可能促使个体通过提示或上下文辅助回忆。
        3. 信心判断（Confidence Judgments）  
            定义：对自己答案或决定的确定程度评估。  
            示例：在选择题中，感觉某个选项非常正确，决定不更改。  
            作用：影响决策，如是否坚持答案或重新检查。
        4. 难度评估（Difficulty Assessments）  
            定义：感知任务的难易程度或所需努力。  
            示例：阅读复杂文章时，感觉需要更多时间理解，决定放慢速度。  
            作用：帮助调整策略，如分解任务或寻求帮助。
        5. 情绪反应（Emotional Responses）  
            定义：学习过程中伴随的情绪，如满足、挫折或焦虑。  
            示例：掌握新概念后感到满足，或因反复失败感到挫折。  
            作用：情绪反应可能触发策略调整，如因挫折而选择更简单的学习方法。
        6. 监控理解（Monitoring Comprehension）  
            定义：在阅读或听讲时检查理解程度。  
            示例：边读边问自己“我明白了吗？”发现不理解后重读。  
            作用：实时调整学习过程，确保理解。
        7.自我提问（Self-Questioning）  
            定义：通过提问检查理解或进展。  
            示例：学习后问自己“这个概念的主要点是什么？”以确认掌握。  
            作用：增强主动学习，识别知识缺口。
        8.反思性思考（Reflective Thinking）  
            定义：任务完成后反思学习过程，评估策略效果。  
            示例：考试后思考哪些复习方法有效，决定下次改进。  
            作用：为未来学习提供反馈，优化方法。

反思过程使用元认知结构分析，反思内容采用结构化思维模型
"""

In [22]:
import os
import ollama
import json

def chat_with_ollama(user_prompt :str, system_prompt :str = None):
            # 调用Ollama处理（带异常重试机制）
    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = ollama.chat(
                model='qwq',
                messages=[
                    {
                    'role': 'system',
                    'content': system_prompt
                    },
                    {
                    'role': 'user',
                    'content': user_prompt
                }],
                options={'num_predict': 9000,
                        'num_ctx': 8000,
                        'temperature': 0.9,},
            )
            print(response['message']['content'])
            return response['message']['content']
        except Exception as e:
            if attempt == max_retries - 1:
                raise RuntimeError(f"Ollama接口调用失败：{str(e)}")
            
def process_corpus_with_ollama():
    corpus_dir = "../data/knowledge/output"

    
    # 元认知自省：验证目录存在性
    if not os.path.exists(corpus_dir):
        raise FileNotFoundError(f"语料目录不存在：{corpus_dir}")
    if not os.path.isdir(corpus_dir):
        raise NotADirectoryError(f"指定路径不是目录：{corpus_dir}")

    # 循环读取文件（按文件名排序）
    file_list = sorted([
        f for f in os.listdir(corpus_dir) 
        if os.path.isfile(os.path.join(corpus_dir, f))
    ])

    
    for filename in file_list:
        outputs = []
        filepath = os.path.join(corpus_dir, filename)
        user_prompt = """下面是一份投研报告，请对报告文本进行插入式修改，添加的内容需遵循元认知结构。
具体要求如下：识别文本中的思考内容，注意务必使用基于元认知结构进行反思，插入基于元认知结构的反思内容。
在标注时，请使用<meat></meta>来插入您的注释，格式为：<meta>...</meta>。
例如，‘我需要先规划在执行’编辑为‘我需要先规划在执行<meta>元认知式反思：在规划前是否对需求足够了解？</meta>’。
注意不能省略报告原文
：\n\n"""
        # 元认知自省：异常捕获与编码处理
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                content = f.read().strip()
                user_prompt += f"<report>{content}\n\n</report>\n\n"
        except UnicodeDecodeError:
            with open(filepath, 'r', encoding='gbk') as f:
                content = f.read().strip()
                user_prompt += f"【{filename}】\n{content}\n\n"
        for ii in range(10):
            # 采样出10个结果
            #outputs.append()
            save_content = {'raw_content': content, 'output': chat_with_ollama(user_prompt, system_prompt), 'filename': filename}
            save_output([save_content])


#把输出的结果按行写入文件
def save_output(outputs
):
    with open("../data/corpus/extrospection/output.txt", "a+", encoding='utf-8') as f:
        for output in outputs:
            # format json string to one line
            output = json.dumps(output, ensure_ascii=False)
            f.write(f"{output}\n")
            


In [23]:
result = process_corpus_with_ollama()


<think>
好的，我现在需要处理用户的请求，对一份投研报告进行插入式修改，添加基于元认知结构的反思内容。用户要求使用<meat></meta>标签，并且不能省略原文。

首先，我得仔细阅读整个报告，理解其主要内容和结构。这份报告详细讲述了桑伯恩地图公司的发展历程、面临的技术挑战以及巴菲特的投资案例分析。接下来，我需要按照元认知结构来反思文本中的思考内容。

根据用户提供的元认知结构，我需要识别三个部分：元认知知识（陈述性、程序性和条件性）、调节（规划、监控、评估）和体验（比如知道感、难度评估等）。我的任务是在适当的位置插入这些反思的注释。

首先，分析报告中的关键点。例如，在描述桑伯恩公司如何应对计分卡技术冲击时，可以考虑元认知知识中的陈述性知识：了解火灾保险行业和技术变化的影响。同时，巴菲特的投资决策可能涉及程序性和条件性知识，比如评估公司的证券组合价值和资产剥离策略。

接下来，在规划部分，当报告提到巴菲特的策略时，我需要插入关于他如何计划通过分拆公司来实现价值的反思。监控方面，可能在分析财务数据时考虑如何跟踪利润变化和调整投资策略。评估则体现在对桑伯恩未来前景的判断上。

元认知体验如困难评估可能会出现在讨论技术冲击对公司的影响时，或者当巴菲特面对复杂财务结构做出决策时的情绪反应（情绪反应）。此外，在巴菲特看到资产组合价值但其他人忽视时，可以插入知道感或信心判断的内容。

需要注意的是，每个插入点必须紧扣原文内容，并且使用正确的元认知术语。例如，当提到桑伯恩公司面临技术冲击时，可能需要反思是否有条件性知识来应对变化，或者评估策略的有效性。

此外，要确保所有插入的<meat>标签正确嵌套在报告文本中，不影响原意。同时，必须保持原文完整，不能遗漏任何段落或数据。

现在开始逐段处理：

例如，在开头部分介绍桑伯恩公司成立时，可以加入元认知知识中的陈述性知识反思，思考当时环境对战略选择的影响。

当讨论火灾保险行业起源时，可能涉及程序性知识如何应用于新市场进入策略。在描述财务数据下滑时，插入监控或评估的反思，如是否定期审查业务指标。

巴菲特的投资分析部分是关键点，需要多处插入元认知内容：资产组合估值（条件性和程序性知识）、分拆决策（规划、评估）以及董事会分歧（条件性知识的应用）。

最后，在讨论公司后续发展时，可以加入反思其适应新技术的能力是否符合元认知调节